# Kamino

This notebook processes Kamino loan states to get to the liquidable debt of the protocol. The data will then used to produce the main chart - liquidable debt vs. available supply.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import collections
import decimal
import logging
import sys

import pandas

sys.path.append('..')

import src.kamino_vault_map
import src.loans.kamino
import src.prices
import src.visualizations.main_chart


logging.basicConfig(level=logging.INFO)

## Compute liquidable debt and available supply at various prices

In [ ]:
import db
import src.loans.loan_state


protocol = 'kamino'
session = db.get_db_session()

current_loan_states = src.loans.loan_state.fetch_loan_states(protocol, session)

In [ ]:
state = src.loans.kamino.KaminoState(
    verbose_users={'HrD37nAyABXZUvmRzQ2JWGxvGgA9jC11cDbkKthmio3V'},
    initial_loan_states=current_loan_states,
)

In [ ]:
collateral = [x.collateral for x in state.loan_entities.values()]
collateral_per_token = collections.defaultdict(decimal.Decimal)
for user_collateral in collateral:
    for token, amount in user_collateral.items():
        collateral_per_token[token] += amount
collateral_tokens = {x for x in collateral_per_token.keys()}

In [ ]:
debt = [x.debt for x in state.loan_entities.values()]
debt_per_token = collections.defaultdict(decimal.Decimal)
for user_debt in debt:
    for token, amount in user_debt.items():
        debt_per_token[token] += amount
debt_tokens = {x for x in debt_per_token.keys()}

In [ ]:
for collateral_token in collateral_tokens:
    current_loan_states[f'collateral_{collateral_token}'] = current_loan_states['collateral'].apply(lambda x: x[collateral_token] if collateral_token in x else decimal.Decimal('0'))
for debt_token in debt_tokens:
    current_loan_states[f'debt_{debt_token}'] = current_loan_states['debt'].apply(lambda x: x[debt_token] if debt_token in x else decimal.Decimal('0'))

In [ ]:
# Get prices.
underlying_collateral_tokens = [
    src.kamino_vault_map.lp_to_mint_map[x]
    for x in collateral_tokens
    if x in src.kamino_vault_map.lp_to_mint_map
]
underlying_debt_tokens = [
    src.kamino_vault_map.supply_vault_to_mint_map[x]
    for x in debt_tokens
    if x in src.kamino_vault_map.supply_vault_to_mint_map
]
token_prices = src.prices.get_prices_for_tokens(underlying_collateral_tokens + underlying_debt_tokens)
token_prices

In [ ]:
collateral_token_parameters = {
    collateral_token: src.kamino_vault_map.lp_to_info_map.get(collateral_token, None)
    for collateral_token
    in collateral_tokens
}
collateral_token_parameters

In [ ]:
debt_token_parameters = {
    debt_token: src.kamino_vault_map.supply_to_info_map.get(debt_token, None)
    for debt_token
    in debt_tokens
}
debt_token_parameters

In [ ]:
for collateral_token in collateral_tokens:
    if not collateral_token_parameters[collateral_token]:
        continue
    if not collateral_token_parameters[collateral_token]['underlying_decs']:
        continue
    decimals = collateral_token_parameters[collateral_token]['underlying_decs']
    ltv = collateral_token_parameters[collateral_token]['ltv']
    underlying_token = src.kamino_vault_map.lp_to_mint_map[collateral_token]
    current_loan_states[f'collateral_usd_{collateral_token}'] = (
        current_loan_states[f'collateral_{collateral_token}'].astype(float)
        / (10**decimals)
        * (ltv/100)
        * token_prices[underlying_token]
    )

for debt_token in debt_tokens:
    if not debt_token_parameters[debt_token]:
        continue
    if not debt_token_parameters[debt_token]['underlying_decs']:
        continue
    decimals = debt_token_parameters[debt_token]['underlying_decs']
    ltv = debt_token_parameters[debt_token]['ltv']
    underlying_token = src.kamino_vault_map.supply_vault_to_mint_map[debt_token]
    current_loan_states[f'debt_usd_{debt_token}'] = (
        current_loan_states[f'debt_{debt_token}'].astype(float)
        / (10**decimals)
        * (1/(ltv/100) if ltv else 1)
        * token_prices[underlying_token]
    )

In [ ]:
# Select tokens and perform computations for this pair.
COLLATERAL_TOKEN = "So11111111111111111111111111111111111111112"  # SOL
DEBT_TOKEN = "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v"  # USDC


mint_to_lp_map = {x: [] for x in src.kamino_vault_map.lp_to_mint_map.values()}
for x, y in src.kamino_vault_map.lp_to_mint_map.items():
    mint_to_lp_map[y].append(x)
mint_to_supply_map = {x: [] for x in src.kamino_vault_map.supply_vault_to_mint_map.values()}
for x, y in src.kamino_vault_map.supply_vault_to_mint_map.items():
    mint_to_supply_map[y].append(x)
collateral_token_price = token_prices[COLLATERAL_TOKEN]



def compute_liquidable_debt_at_price(
    loan_states: pandas.DataFrame,
    token_prices: dict[str, float],
    debt_token_parameters: dict[str, float],
    collateral_token: str,
    target_collateral_token_price: decimal.Decimal,
    debt_token: str,
) -> decimal.Decimal:
    lp_collateral_tokens = mint_to_lp_map[collateral_token]
    supply_collateral_tokens = mint_to_supply_map[collateral_token]

    price_ratio = target_collateral_token_price / token_prices[collateral_token]
    for lp_collateral_token in lp_collateral_tokens:
        lp_collateral_column = f'collateral_usd_{lp_collateral_token}'
        if lp_collateral_column in loan_states.columns:
            loan_states[lp_collateral_column] = loan_states[lp_collateral_column] * price_ratio
    for supply_collateral_token in supply_collateral_tokens:
        supply_collateral_column = f'debt_usd_{supply_collateral_token}'
        if supply_collateral_column in loan_states.columns:
            loan_states[supply_collateral_column] = loan_states[supply_collateral_column] * price_ratio
    loan_states['collateral_usd'] = loan_states[[x for x in loan_states.columns if 'collateral_usd_' in x]].sum(axis = 1)
    loan_states['debt_usd'] = loan_states[[x for x in loan_states.columns if 'debt_usd_' in x]].sum(axis = 1)

    loan_states['loan_to_value'] = loan_states['debt_usd'] / loan_states['collateral_usd']
    liquidation_parameters = debt_token_parameters[supply_collateral_tokens[0]]
    liquidation_threshold = (
        liquidation_parameters['liquidation_threshold_pct'] / 100
        if supply_collateral_tokens[0] in debt_token_parameters
        else 0.5
    )
    loan_states['liquidable'] = loan_states['loan_to_value'] < liquidation_threshold
    # 20% of the debt value is liquidated.
    liquidable_debt_ratio = 0.2 * (
        liquidation_parameters['min_liquidation_bonus_bps']
        if supply_collateral_tokens[0] in debt_token_parameters
        else 0.02
    )
    loan_states['debt_to_be_liquidated'] = liquidable_debt_ratio * loan_states['debt_usd'] * loan_states['liquidable']
    return loan_states['debt_to_be_liquidated'].sum()



data = pandas.DataFrame(
    {
        "collateral_token_price": src.visualizations.main_chart.get_token_range(collateral_token_price),
    }
)
data

# TODO: Compute liqidable debt.
data['liquidable_debt'] = data['collateral_token_price'].apply(
    lambda x: compute_liquidable_debt_at_price(
        loan_states = current_loan_states.copy(),
        token_prices = token_prices,
        debt_token_parameters = debt_token_parameters,
        collateral_token = COLLATERAL_TOKEN,
        target_collateral_token_price = x,
        debt_token = DEBT_TOKEN,
    )
)
data['liquidable_debt_at_interval'] = data['liquidable_debt'].diff().abs()
data.dropna(inplace = True)